In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
trials_data= pd.read_csv('trials_training_data.csv')

In [3]:
trials_data

,Name,State,Notes,User,Tags,Created,Runtime,Sweep,base_lr,epochs,...,loss/val,lr,precip_val_loss,temp_val_loss,tmax_val_loss,tmin_val_loss,total_val_loss,trial,weights,initial_weights
0,trial_0,finished,-,NaN,NaN,2025-07-20T10:31:40.000Z,17670,NaN,NaN,NaN,...,0.000054,0.0008,0.000004,0.000056,0.000084,0.000098,0.000054,0,"[0.2886978837916178,0.3378427925582116,0.16448...",NaN
1,trial_10,finished,-,NaN,NaN,2025-07-21T04:27:06.000Z,7075,NaN,NaN,NaN,...,0.000060,0.0008,0.000004,0.000077,0.000094,0.000080,0.000057,24,"[0.33606969326889924,0.13119827552117166,0.336...",NaN
2,trial*_8,finished,"From the new run, it starts here",NaN,NaN,2025-07-21T17:40:41.000Z,22199,NaN,NaN,NaN,...,0.000060,0.0010,0.000005,0.000075,0.000079,0.000107,0.000059,8,"[0.3334388539928033,0.1684442201994812,0.21004...","[0.9903706241712326,0.500308243923479,0.623858..."
3,trial_11,finished,-,NaN,NaN,2025-07-21T02:28:13.000Z,7129,NaN,NaN,NaN,...,0.000059,0.0010,0.000004,0.000063,0.000093,0.000090,0.000059,21,"[0.26937599450155403,0.30615865716140617,0.252...",NaN
4,trial_3,finished,-,NaN,NaN,2025-07-20T15:36:19.000Z,7301,NaN,NaN,NaN,...,0.000062,0.0010,0.000005,0.000066,0.000114,0.000103,0.000059,4,"[0.36503006197686494,0.2575344108920214,0.2530...",NaN
5,trial_14,finished,-,NaN,NaN,2025-07-21T10:22:04.000Z,7038,NaN,NaN,NaN,...,0.000054,0.0008,0.000005,0.000065,0.000101,0.000091,0.000052,31,"[0.39917137031543554,0.23347669290918308,0.232...",NaN
6,trial*_21,finished,-,NaN,NaN,2025-07-22T09:35:14.000Z,9259,NaN,NaN,NaN,...,0.000065,0.0008,0.000004,0.000072,0.000068,0.000115,0.000057,21,"[0.29526200309812994,0.17086641653602871,0.159...","[0.7612033940568413,0.440504009431679,0.410372..."
7,trial*_16,finished,-,NaN,NaN,2025-07-22T05:53:19.000Z,13301,NaN,NaN,NaN,...,0.000055,0.0010,0.000004,0.000057,0.000082,0.000108,0.000055,16,"[0.2905324276569757,0.33912427471113427,0.1544...","[0.5642289734041632,0.6585968489638253,0.29992..."
8,trial_5,finished,-,NaN,NaN,2025-07-20T17:38:06.000Z,7075,NaN,NaN,NaN,...,0.000051,0.0008,0.000005,0.000058,0.000072,0.000087,0.000051,8,"[0.3118304899680882,0.2532597408016314,0.20394...",NaN
9,trial_4,finished,The runtime is less because the run was initia...,NaN,NaN,2025-07-19T16:53:35.000Z,6,NaN,0.0001,20.0,...,0.000058,0.0008,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN


In [4]:
trials_data.columns

Index(['Name', 'State', 'Notes', 'User', 'Tags', 'Created', 'Runtime', 'Sweep',
       'base_lr', 'epochs', 'loss', 'loss_weights', 'max_lr', 'mode',
       'optimizer', 'optuna_trial', 'scheduler', 'RhiresD/train',
       'RhiresD/val', 'TabsD/train', 'TabsD/val', 'TmaxD/train', 'TmaxD/val',
       'TminD/train', 'TminD/val', 'best_val_loss',
       'best_val_loss_per_channel', 'epoch', 'epoch_time', 'loss/train',
       'loss/val', 'lr', 'precip_val_loss', 'temp_val_loss', 'tmax_val_loss',
       'tmin_val_loss', 'total_val_loss', 'trial', 'weights',
       'initial_weights'],
      dtype='object')

In [ ]:
x = trials_data['RhiresD/val'].values
y = trials_data['best_val_loss'].values

def pareto_front_2d(x, y):
    # Sort by x (first objective)
    idx_sorted = np.argsort(x)
    pareto_idx = []
    min_y = np.inf
    for idx in idx_sorted:
        if y[idx] < min_y:
            pareto_idx.append(idx)
            min_y = y[idx]
    pareto_points = np.array(list(zip(x[pareto_idx], y[pareto_idx])))
    return pareto_points, np.array(pareto_idx)

pareto_points, pareto_idx = pareto_front_2d(x, y)

plt.figure(figsize=(12, 8))
plt.scatter(x, y, label='Trials', color='blue', alpha=0.5, s=30)
plt.plot(pareto_points[:,0], pareto_points[:,1], color='red', linewidth=3, label='Pareto Front')

# Annotate Pareto "elbow" (closest to origin)
pareto_distances = np.sqrt(pareto_points[:,0]**2 + pareto_points[:,1]**2)
elbow_idx = np.argmin(pareto_distances)
elbow_x, elbow_y = pareto_points[elbow_idx]
elbow_trial_idx = pareto_idx[elbow_idx]
elbow_trial_name = trials_data.iloc[elbow_trial_idx]['Name']
plt.scatter(elbow_x, elbow_y, marker='*', s=250, color='green', label='Pareto elbow')
plt.annotate(f'{elbow_trial_name}', (elbow_x, elbow_y),
             textcoords="offset points", xytext=(10,-20), ha='left', color='green', fontsize=11, fontweight='bold')

plt.xlabel('Precipitation channel validation loss')
plt.ylabel('Best total validation loss')
plt.title('Pareto Front: Total validation loss vs Precipitation Val Loss for "constrained" trials')
plt.legend()
plt.grid(False)
plt.tight_layout()
plt.savefig('pareto_front_plot.png', dpi=500, bbox_inches='tight')